In [1]:
# 0.3 수정사항
# try except 및 failed dict 추가

# ===================================
# 0.31.10 2020.09.29 # 작업자: 김성근 
# ===================================
# ===================================
# 0.31.10 2020.10.05 # 작업자: 김성근 
# ===================================


In [2]:
# ===============================
# NAVER News Web Crawler _  
# ===============================

# ---------------------
# * Process 
# ---------------------
# Step0: "Preparation" _ Import Packages 
# Step1: "Setting" _ Set File Directory & Parameters
# Step2: "Crawling" 
#     2.1) Open Blank Browser
#     2.2) Crawling News contents
#         2.2.1) Serach & Get news list
#         2.2.2) Collect News Contents
# Step3: "Reporting" _ Summarise the programme result 

# ---------------------
# * I/O 
# ---------------------
# Input : 
#     1. [keywords.csv] > Keywords = [keyword1, keyword2, ...]
#     2. Crawling term: 2 parameters [_date_start, _date_end](format="YYYY.MM.DD")
# Output : 
#     1. "naver_news_crawling_df1.csv" (df1): Contains [keywords, period, title, press, url]


In [3]:
# ===============================
# Step0: "Preparation" _ Import Packages 
# ===============================
import requests
import time
from itertools import count
import datetime as dt
import time

# from multiprocessing import Pool # MultiProcessing 
# pool = Pool(processes=4)

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [4]:
# ================================
# Step1: "Setting" _ Set File Directory & Parameters
# ================================

# set file dir. 
team_dir = r'D:/02.users/yjin123/team3/' 
code_dir = r'D:/02.users/yjin123/team3/euro/euro_code/' 
data_dir = r'D:/02.users/yjin123/team3/euro/euro_data/' 
save_dir = r'D:/02.users/yjin123/team3/euro/euro_data/news_data/' 
logg_dir = r'D:/02.users/yjin123/team3/euro/euro_data/logger/' 

# set browser driver dir.
chrome_dir = team_dir+'chromedriver_win32 (1)/chromedriver'

# set Keyword dir.
#keywords_dir = 'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_data/keywords/'
#keywords_file = 'Z:/0_프로젝트/데이터바우처 2020/POC/3팀(박재인PL)/03 유로인스트루먼츠/code/김성근임시/euro_data/keywords/keywords_test_20200910.csv'

# Load target search keywords 
keywords_df = pd.read_csv('keywords.csv',encoding='euc-kr')
keywords_list = keywords_df['keyword'].tolist()

# set target date range
_date_start_list = pd.date_range('2015.01.01',periods=6,freq='AS-JAN').strftime('%Y.%m.%d')
_date_end_list = pd.date_range('2015.12.31',periods=6,freq='A-DEC').strftime('%Y.%m.%d')
# target_date = _date_start[:4]

# ### debug note ==================================
# : 월, 분기 단위 loop를 받는 것, 

# _date_start = '2010.01.01'
# _date_end = '2016.12.31'

# _date_start_list = pd.date_range(_date_start, _date_end, freq='MS').strftime('%Y.%m.%d')
# _date_end_list = pd.date_range(_date_start, _date_end, freq='M').strftime('%Y.%m.%d')

# print(_date_start_list)
# print(_date_end_list)
# ### =============================================

# Today, now 
time_start = dt.datetime.now()

_today=time_start.date().strftime('%Y.%m.%d')
_time_start=time_start.time().strftime('%H:%M:%S')

# Set Open Page URL
origin_url='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EB%84%A4%EC%9D%B4%EB%B2%84+%EB%89%B4%EC%8A%A4'

# Set dir. for file saving
df1_name_head = "naver_news_crawling_df1"
df2_name_head = "naver_news_crawling_df2"

## Assign headless setting 
# headless_mode = True 
# if headless_mode :
#     options = webdriver.ChromeOptions()
#     options.add_argument('headless')    # add 'Headless' 
#     options.add_argument('window-size=1920x1080')    # set current display setting 
#     options.add_argument("disable-gpu") 
# ### debug note ==================================
# : Chrome의 Headless option이 ignored 되는 이슈. >>> (pressumed to be) Deprecated 
# >>> 없에는 걸로 
# ### =============================================

# Write Log
verbose_ = True    # Print log in console 

# # Write log in logger.txt
# logger_name = '1-01_logger_{0}_{1}.txt'.format(str(_today).replace('.',''), str(_time_start).replace(':',''))   
# logg = open(logg_dir+logger_name,'w')
    
    

In [5]:
# Step1: Setting Report 

start_report = """
# ================================  
# NAVER News Crawler _  
# ================================  

* Programme execution  
Date: {_today}  
Time start: {_time_start}
  
* Crawling parameters  
Date start : {_date_start}  
Date end   : {_date_end}  
The number of keywords : {len_keywords}  
"""

end_report = """
# ================================  
# Report for Process _  
# ================================  

* Programme Execution  
End of Process: {_time_end} 
Time Spent: {_time_spent}  
"""

### *** 기업명 색인 

In [6]:
keywords_list = keywords_df['keyword'].tolist()

In [7]:
# keywords_list.index('삼보모터스')
corp_from_ = '서울옥션' # ex:) '삼보모터스' 
trunc_list = keywords_list[keywords_list.index(corp_from_):] # ex:) "삼보모터스"부터 시작하는 기업목록 반환 
# trunc_list[:5]

In [8]:
trunc_list

['서울옥션',
 '쏠리드',
 '에이스일렉',
 '아인스M&M',
 '한통데이타',
 '에스피코프',
 '굿이엠지',
 '올리브나인',
 'GK파워',
 '투미비티',
 '야호',
 '케이에스알',
 '보광티에스',
 '케이디세코',
 '엠텍비젼에셋',
 '케이이엔지',
 '산양전기',
 '아이팩토리',
 '세화아이엠씨',
 '현대EP',
 '용평리조트',
 'AJ네트웍스',
 '쌍용머티리얼',
 '코리아오토글라스',
 '텔코웨어',
 '두산인프라코어',
 '대우조선해양',
 '휴비스',
 '동원F&B',
 '아이마켓코리아',
 '대우건설',
 '포스코대우',
 '아비스타',
 '에이블씨엔씨',
 '팬오션',
 '뉴프렉스',
 'MDS테크',
 '덕신하우징',
 '모두투어',
 '서원인텍',
 '스틸플라워',
 '아바텍',
 '자연과환경',
 '와토스코리아',
 '젬백스',
 '티엘아이',
 '옴니시스템',
 '비에이치아이',
 '옵트론텍',
 '오스코텍',
 '파버나인',
 '링네트',
 '아리온',
 '쎄트렉아이',
 '우리이티아이',
 '엘비세미콘',
 '아바코',
 '휘닉스소재',
 '제이엔케이히터',
 '동부라이텍',
 '시큐브',
 '대봉엘에스',
 '현대아이비티',
 '멀티캠퍼스',
 '아이씨디',
 '와이비엠넷',
 '네오티스',
 '아이크래프트',
 '하이텍팜',
 '필링크',
 '엔텔스',
 '웹젠',
 '가비아',
 '덕산하이메탈',
 '나노신소재',
 '유니온커뮤니티',
 '메디포스트',
 '홈캐스트',
 'EMW',
 '넥스턴',
 '메가스터디',
 '에프알텍',
 'iMBC',
 '바이오톡스텍',
 '위메이드',
 '크리스탈',
 '셀트리온제약',
 '엘앤에프',
 '큐에스아이',
 '이상네트웍스',
 '아이컴포넌트',
 '코위버',
 'KG모빌리언스',
 '플랜티넷',
 'CSA 코스믹',
 '디지털대성',
 '이노와이어리스',
 '제주반도체',
 '유진테크',
 '한국전자금융',
 '대성파인텍',
 '테라셈',
 '유비쿼스홀딩스'

### *** 함수 정의 

In [9]:
def crawlingNewsContents(keyword,_date_start,_date_end):
    # ================================
    #     2.2) Crawling News contents
    # ================================
    # html=driver.page_source
    target_date=_date_start[:4]
    # ================================
    #     2.2.1) Serach & Get news list
    # ================================
    # Assign blank lists 
    completed_keywords=[]
    keywords=[]
    periods=[]
    titles=[]
    presses=[]
    urls=[]

    #  keyword report
    get_report = """
    * Keyword ({idx}/{tot}): [{keyword}]
    """.format(keyword=keyword, idx=i+1, tot=len(keywords_list))
    print(get_report)
    logg.write(get_report+'\n') 

    # 검색과정
    keyword_search=f'"{keyword}"'
    search=driver.find_element_by_xpath('//*[@id="nx_query"]') # 검색창 선택
    search.clear()  
    search.send_keys(keyword_search) # 검색어 입력
    search=driver.find_element_by_xpath('//*[@id="nx_search_form"]/fieldset/button').click() # 검색 클릭
    
    try:
        driver.implicitly_wait(1)
        option=driver.find_element_by_class_name('bt_option').click() # 검색옵션 클릭
        period_option=driver.find_element_by_xpath('//*[@id="snb"]/div/ul/li[2]/a').click() # 기간 탭 클릭
    except:
        option=driver.find_element_by_class_name('bt_option').click() # 검색옵션 클릭
        period_option=driver.find_element_by_xpath('//*[@id="snb"]/div/ul/li[2]/a').click() # 기간 탭 클릭

    # Wait until Dropdown filter loaded
    element=WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID,'news_input_period_begin')))
    # Enter target date to search 
    date_head=driver.find_element_by_xpath('//*[@id="news_input_period_begin"]')
    date_head = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="news_input_period_begin"]')))  
    date_head.clear() 
    date_head.send_keys(_date_start)
    date_tail=driver.find_element_by_xpath('//*[@id="news_input_period_end"]')
    date_tail.clear()
    date_tail.send_keys(_date_end)
    apply=driver.find_element_by_class_name('_btn_submit').click()

    # 데이터 프레임 생성 - 검색어, 기간, 기사제목, 언론사, url        
    p=1   
    while True:
        # Parsing the current result page 
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser') 
        news_list=soup.find('ul',class_='type01') 
        # Get news (unless there's no news)
        try: 
            news_titles=news_list.find_all('dl')   # get each news elements
        except AttributeError as err1 : # err1: If there's no news to collect >>> skip the keyword 
            get_report = "    *** Error 2.2.1. : [{0}]_[{1}]: {2} ".format(keyword, target_date, err1)
            if verbose_ : print(get_report) 
            #print(get_report)
            logg.write(get_report+'\n') 

            break 

        # Extract page source 
        for dl in news_titles:
            d = _date_start[:4]

            title = dl.find('dt').get_text()  # 기사 제목
            press = dl.find('dd',class_='txt_inline').find('span').get_text() # 언론사
            url = dl.find('dd').find('a').get('href') # url

            keywords.append(keyword)
            periods.append(d)
            titles.append(title)
            presses.append(press)
            urls.append(url)

        # 다음페이지
        try:
            next_p=driver.find_element_by_class_name('next').click()
            p+=1
            
            # +v.31.20) 페이지 넘어가는 수가 과다하게 잡힐 때,(네이버의 TimeOut Exception으로 추정 중)
            # 400 페이지가 넘어가면 수집 불가, 만약 페이지 수가 500 이상 도달한 (이상)경우에 
            if p > 500 : raise # 강제 exception 반환 
            
        except:
            get_report = f'[{keyword}]_[{d}]:  마지막 {p} 페이지입니다.'
            #if verbose_ : print(get_report)
            logg.write(get_report+'\n') 
            # +v.31.01) 페이지가 초과되어 더이상 기사 수집이 안된 경우, log, console에 추가 print 
            if p == 400 : 
                page_max_report = f"   *** [{keyword}]: 400 페이지 초과로 추가 기사 load failed "  
                print(page_max_report)
                logg.write(page_max_report+'\n') 
            # +v.31.20) 페이지 넘어가는 수가 과다하게 잡힐 때
            elif p > 500 : 
                page_max_report = f"    *** Error 2.2.1.[{keyword}]: 비정상 페이지 Count 발생 "  
                print(page_max_report)
                logg.write(page_max_report+'\n') 
            break
        continue

    # Export News List DataFrame For each keywords (For whole period)
    # 데이터 프레임 생성 - 1. 검색어, 기간, 제목, 언론사, url
    mCols = []
    df_1 = pd.DataFrame(mCols)

    df_1['keyword'] = keywords
    df_1['period'] = periods
    df_1['title'] = titles
    df_1['press'] = presses
    df_1['url'] = urls

    # drop duplicated articles with 'urls'
    df_1.drop_duplicates(['url'],keep='first',inplace=True)
    # +v.31.01) without 'sports' articles 
    indexNames=df_1[df_1['url'].isin(['sports.naver'])].index
    df_1.drop(indexNames,inplace=True)
    # +v.31.01) without 'entertain' articles 
    indexNames=df_1[df_1['url'].isin(['entertain.naver'])].index
    df_1.drop(indexNames,inplace=True)

    # export news list DataFrame to csv
    save_filename = save_dir+df1_name_head+'_{0}_from{1}_to{2}.csv'.format(keyword,_date_start,_date_end)
    completed_keywords.append(keyword)

    try : 
        df_1.to_csv(save_filename, index=False)
        table_report = """
 >>> [{keyword}]_ News List(df1): Successfully Saved  
       File name: {_file_name} , {_naver_news} NAVER News URLs have been collected
        """.format(keyword=keyword,
                   _file_name=save_filename.split('/')[-1],
                   _naver_news=df_1['url'].ne('#').sum(), 
                   _searched_news=df_1.shape[0])
        if verbose_ : print(table_report)
        logg.write(table_report+'\n') 
    except Exception as e : 
        table_report = """
    *** Error 2.2.1. :Error has been occurred: {_err}
          Keyword: [{keyword}]""".format(_err=e, keyword=keyword)
        print(table_report)
        logg.write(table_report+'\n')


#     # close browser
#     driver.close()


### *** 크롤링 수행 

In [10]:
# # proc 3. Loop
# # ================== ver. 0.31.20 ==================

# # 크롬 드라이버 인스턴스 생성
# driver=webdriver.Chrome(chrome_dir)

# # Enter "Naver Search" page
# driver.get(origin_url)
# news=driver.find_element_by_class_name('lnb4').click() # 뉴스탭 클릭

# # for exception tracking
# failed_dict={}

# # for 1): loop for target periods ================================
# for _date_start,_date_end in zip(_date_start_list,_date_end_list): 
#     # "한 단위 주기"를 받습니다 ex:) 2010.01.01 ~ 2010.12.31    
#     print("\n *** Search Periods from : {0} ~ to : {1} ". format(_date_start,_date_end))
    
#     # Write log in logger.txt (log for each target period)
#     logger_name = '1-01_logger_{0}_{1}.txt'.format(str(_today).replace('.',''), str(_time_start).replace(':',''))   
#     logg = open(logg_dir+logger_name,'w')
    
#     # 한 단위의 기간별로 로그 생성 >>> 시작지점 로그 기록 
#     get_report = start_report.format(_today=_today, 
#            _time_start=_time_start, 
#            _date_start=_date_start, 
#            _date_end=_date_end, 
#            len_keywords=len(keywords_list))
#     if verbose_ : print(get_report)
#     logg.write(get_report+'\n') 

#     # for exception tracking, append 'failed loop keywords'
#     failed_list_yearly=[]
    
#     #for i,keyword in enumerate(keywords_list):
#     for i,keyword in enumerate(trunc_list if _date_start[:4]=='2010' else keywords_list):  
#         try:
#             crawlingNewsContents(keyword,_date_start,_date_end)    
#         except:
#             # 개별 기업 크롤링 수행중 에러 발생시 
#             #print("    **** err proc 3. in except : {0} - {1} ".format(keyword, _date_start[:4]))
#             failed_list_yearly.append(keyword)
#             failed_dict[_date_start[:4]]=failed_list_yearly
#             print("    **** err proc 3. {0}".format(failed_dict))
            
#             # re-boot browser ===== # v.31.20 debug(1)
#             driver.close()     # close current browser first.
#             time.sleep(600)     # 10분 일시정지 (크롤링 차단 우회) 
#             driver.get(origin_url)     # open new browser
#             news=driver.find_element_by_class_name('lnb4').click() # 뉴스탭 클릭
            
#             continue
            
            
#     # ================================
#     # Step3: "Reporting" _ Summarise the programme result 
#     # ================================
#     # 한 단위 기간 마친 후, 완료 로그 기록 
#     time_end = dt.datetime.now() # v.31.20 debug
#     get_report = end_report.format(_time_end=time_end.time().strftime('%H:%M:%S'),
#           _time_spent=str(time_end - time_start).split('.')[0])
#     if verbose_ : print(get_report)
#     logg.write(get_report+'\n') 
#     # 로그 종료 
#     logg.close()

# # close browser
# driver.close()

In [ ]:
# proc 3. Loop
# ================== ver. 0.31.20 ==================

# 크롬 드라이버 인스턴스 생성
driver=webdriver.Chrome(chrome_dir)

# Enter "Naver Search" page
driver.get(origin_url)
news=driver.find_element_by_class_name('lnb4').click() # 뉴스탭 클릭

# for exception tracking
failed_dict={}

# for 1): loop for target periods ================================
for _date_start,_date_end in zip(_date_start_list,_date_end_list): 
    # "한 단위 주기"를 받습니다 ex:) 2010.01.01 ~ 2010.12.31    
    print("\n *** Search Periods from : {0} ~ to : {1} ". format(_date_start,_date_end))
    
    # Write log in logger.txt (log for each target period)
    logger_name = '1-01_logger_{0}_{1}.txt'.format(str(_today).replace('.',''), str(_time_start).replace(':',''))   
    logg = open(logg_dir+logger_name,'w')
    
    # 한 단위의 기간별로 로그 생성 >>> 시작지점 로그 기록 
    get_report = start_report.format(_today=_today, 
           _time_start=_time_start, 
           _date_start=_date_start, 
           _date_end=_date_end, 
           len_keywords=len(keywords_list))
    if verbose_ : print(get_report)
    logg.write(get_report+'\n') 

    # for exception tracking, append 'failed loop keywords'
    failed_list_yearly=[]
    
    #for i,keyword in enumerate(keywords_list):
    for i,keyword in enumerate(trunc_list if _date_start[:4]=='2015' else keywords_list):  
        try:
            crawlingNewsContents(keyword,_date_start,_date_end)    
        except:
            # 개별 기업 크롤링 수행중 에러 발생시 
            #print("    **** err proc 3. in except : {0} - {1} ".format(keyword, _date_start[:4]))
            failed_list_yearly.append(keyword)
            failed_dict[_date_start[:4]]=failed_list_yearly
            print("    **** err proc 3. {0}".format(failed_dict))
            
            # re-boot browser ===== # v.31.20 debug(1)
            driver.close()     # close current browser first.
            time.sleep(600)     # 10분 일시정지 (크롤링 차단 우회) 
            driver=webdriver.Chrome(chrome_dir)
            driver.get(origin_url)     # open new browser
            news=driver.find_element_by_class_name('lnb4').click() # 뉴스탭 클릭
            
            continue
            
            
    # ================================
    # Step3: "Reporting" _ Summarise the programme result 
    # ================================
    # 한 단위 기간 마친 후, 완료 로그 기록 
    time_end = dt.datetime.now() # v.31.20 debug
    get_report = end_report.format(_time_end=time_end.time().strftime('%H:%M:%S'),
          _time_spent=str(time_end - time_start).split('.')[0])
    if verbose_ : print(get_report)
    logg.write(get_report+'\n') 
    # 로그 종료 
    logg.close()

# close browser
driver.close()


 *** Search Periods from : 2015.01.01 ~ to : 2015.12.31 

# ================================  
# NAVER News Crawler _  
# ================================  

* Programme execution  
Date: 2020.10.20  
Time start: 18:18:02
  
* Crawling parameters  
Date start : 2015.01.01  
Date end   : 2015.12.31  
The number of keywords : 2284  


    * Keyword (1/2284): [서울옥션]
    

 >>> [서울옥션]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_서울옥션_from2015.01.01_to2015.12.31.csv , 1314 NAVER News URLs have been collected
        

    * Keyword (2/2284): [쏠리드]
    

 >>> [쏠리드]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_쏠리드_from2015.01.01_to2015.12.31.csv , 702 NAVER News URLs have been collected
        

    * Keyword (3/2284): [에이스일렉]
    
    *** Error 2.2.1. : [에이스일렉]_[2015]: 'NoneType' object has no attribute 'find_all' 

 >>> [에이스일렉]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에이스일렉_from201


 >>> [에이블씨엔씨]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에이블씨엔씨_from2015.01.01_to2015.12.31.csv , 1835 NAVER News URLs have been collected
        

    * Keyword (35/2284): [팬오션]
    

 >>> [팬오션]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_팬오션_from2015.01.01_to2015.12.31.csv , 1315 NAVER News URLs have been collected
        

    * Keyword (36/2284): [뉴프렉스]
    

 >>> [뉴프렉스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_뉴프렉스_from2015.01.01_to2015.12.31.csv , 136 NAVER News URLs have been collected
        

    * Keyword (37/2284): [MDS테크]
    

 >>> [MDS테크]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_MDS테크_from2015.01.01_to2015.12.31.csv , 306 NAVER News URLs have been collected
        

    * Keyword (38/2284): [덕신하우징]
    

 >>> [덕신하우징]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_덕신하우징_from2015.01.01_to


 >>> [엔텔스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_엔텔스_from2015.01.01_to2015.12.31.csv , 396 NAVER News URLs have been collected
        

    * Keyword (72/2284): [웹젠]
    
   *** [웹젠]: 400 페이지 초과로 추가 기사 load failed 

 >>> [웹젠]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_웹젠_from2015.01.01_to2015.12.31.csv , 2508 NAVER News URLs have been collected
        

    * Keyword (73/2284): [가비아]
    

 >>> [가비아]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_가비아_from2015.01.01_to2015.12.31.csv , 477 NAVER News URLs have been collected
        

    * Keyword (74/2284): [덕산하이메탈]
    

 >>> [덕산하이메탈]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_덕산하이메탈_from2015.01.01_to2015.12.31.csv , 276 NAVER News URLs have been collected
        

    * Keyword (75/2284): [나노신소재]
    

 >>> [나노신소재]_ News List(df1): Successfully Saved  
       File name: naver_news_cr

    *** Error 2.2.1. : [세한엔에스브이]_[2015]: 'NoneType' object has no attribute 'find_all' 

 >>> [세한엔에스브이]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_세한엔에스브이_from2015.01.01_to2015.12.31.csv , 0 NAVER News URLs have been collected
        

    * Keyword (108/2284): [에스제이케이]
    

 >>> [에스제이케이]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에스제이케이_from2015.01.01_to2015.12.31.csv , 1 NAVER News URLs have been collected
        

    * Keyword (109/2284): [루멘스]
    

 >>> [루멘스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_루멘스_from2015.01.01_to2015.12.31.csv , 360 NAVER News URLs have been collected
        

    * Keyword (110/2284): [인프라웨어]
    

 >>> [인프라웨어]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_인프라웨어_from2015.01.01_to2015.12.31.csv , 438 NAVER News URLs have been collected
        

    * Keyword (111/2284): [팅크웨어]
    

 >>> [팅크웨어]_ News List(


 >>> [세아홀딩스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_세아홀딩스_from2015.01.01_to2015.12.31.csv , 181 NAVER News URLs have been collected
        

    * Keyword (142/2284): [해태제과식품]
    

 >>> [해태제과식품]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_해태제과식품_from2015.01.01_to2015.12.31.csv , 24 NAVER News URLs have been collected
        

    * Keyword (143/2284): [엔에스쇼핑]
    

 >>> [엔에스쇼핑]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_엔에스쇼핑_from2015.01.01_to2015.12.31.csv , 502 NAVER News URLs have been collected
        

    * Keyword (144/2284): [STX중공업]
    

 >>> [STX중공업]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_STX중공업_from2015.01.01_to2015.12.31.csv , 537 NAVER News URLs have been collected
        

    * Keyword (145/2284): [케이티스]
    

 >>> [케이티스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_케이티스_from20

...


 >>> [하이셈]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_하이셈_from2015.01.01_to2015.12.31.csv , 89 NAVER News URLs have been collected
        

    * Keyword (503/2284): [티케이케미칼]
    

 >>> [티케이케미칼]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_티케이케미칼_from2015.01.01_to2015.12.31.csv , 333 NAVER News URLs have been collected
        

    * Keyword (504/2284): [코이즈]
    

 >>> [코이즈]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_코이즈_from2015.01.01_to2015.12.31.csv , 158 NAVER News URLs have been collected
        

    * Keyword (505/2284): [인크로스]
    

 >>> [인크로스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_인크로스_from2015.01.01_to2015.12.31.csv , 383 NAVER News URLs have been collected
        

    * Keyword (506/2284): [서진시스템]
    

 >>> [서진시스템]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_서진시스템_from2015.01.01_to20


 >>> [와이솔]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_와이솔_from2015.01.01_to2015.12.31.csv , 544 NAVER News URLs have been collected
        

    * Keyword (539/2284): [글로벌에스엠]
    

 >>> [글로벌에스엠]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_글로벌에스엠_from2015.01.01_to2015.12.31.csv , 329 NAVER News URLs have been collected
        

    * Keyword (540/2284): [와이엠씨]
    

 >>> [와이엠씨]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_와이엠씨_from2015.01.01_to2015.12.31.csv , 36 NAVER News URLs have been collected
        

    * Keyword (541/2284): [에스티팜]
    

 >>> [에스티팜]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에스티팜_from2015.01.01_to2015.12.31.csv , 77 NAVER News URLs have been collected
        

    * Keyword (542/2284): [에이치엔티]
    

 >>> [에이치엔티]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에이치엔티_from2015.01.01_to


 >>> [유테크]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_유테크_from2015.01.01_to2015.12.31.csv , 256 NAVER News URLs have been collected
        

    * Keyword (574/2284): [필옵틱스]
    

 >>> [필옵틱스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_필옵틱스_from2015.01.01_to2015.12.31.csv , 77 NAVER News URLs have been collected
        

    * Keyword (575/2284): [에프엔씨엔터]
    

 >>> [에프엔씨엔터]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_에프엔씨엔터_from2015.01.01_to2015.12.31.csv , 763 NAVER News URLs have been collected
        

    * Keyword (576/2284): [파인테크닉스]
    

 >>> [파인테크닉스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_파인테크닉스_from2015.01.01_to2015.12.31.csv , 524 NAVER News URLs have been collected
        

    * Keyword (577/2284): [휴맥스]
    

 >>> [휴맥스]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_휴맥스_from2015.01.01_t

   *** [한국타이어]: 400 페이지 초과로 추가 기사 load failed 

 >>> [한국타이어]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_한국타이어_from2015.01.01_to2015.12.31.csv , 1493 NAVER News URLs have been collected
        

    * Keyword (610/2284): [애경유화]
    

 >>> [애경유화]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_애경유화_from2015.01.01_to2015.12.31.csv , 177 NAVER News URLs have been collected
        

    * Keyword (611/2284): [동일고무벨트]
    

 >>> [동일고무벨트]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_동일고무벨트_from2015.01.01_to2015.12.31.csv , 150 NAVER News URLs have been collected
        

    * Keyword (612/2284): [한국콜마]
    

 >>> [한국콜마]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_한국콜마_from2015.01.01_to2015.12.31.csv , 1867 NAVER News URLs have been collected
        

    * Keyword (613/2284): [파마리서치프로덕트]
    

 >>> [파마리서치프로덕트]_ News List(df1): Successfully Saved  
  


 >>> [이에스브이]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_이에스브이_from2015.01.01_to2015.12.31.csv , 66 NAVER News URLs have been collected
        

    * Keyword (646/2284): [녹십자랩셀]
    

 >>> [녹십자랩셀]_ News List(df1): Successfully Saved  
       File name: naver_news_crawling_df1_녹십자랩셀_from2015.01.01_to2015.12.31.csv , 71 NAVER News URLs have been collected
        

    * Keyword (647/2284): [더블유게임즈]
    
